# 推薦システムと行列分解

今回は代表的なレコメンデーションシステムの手法として行列因子分解について説明し、Pytorchのモジュール群を使用してどのように実装するのか学習していきます。



リスト6.1　DatasetとDataLoaderの作成（解凍したraiting.csvのあるパスを設定）

In [2]:
import torch
from torch import nn, optim
from torch.utils.data import (Dataset, 
                              DataLoader,
                              TensorDataset)
import tqdm

/home/tk/dev/tokai_teio/inspections/pytorch/.venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
DATA_DIR = '../data/ml-20m'

In [7]:
import pandas as pd
# 訓練データとテストデータを分けるのに使用する
from sklearn import model_selection
df = pd.read_csv(f"{DATA_DIR}/ratings.csv")
# Xは(userId, movieId)のペア
X = df[["userId", "movieId"]].values
Y = df["rating"].values
# 訓練データとテストデータを9:1に分割
train_X, test_X, train_Y, test_Y\
    = model_selection.train_test_split(X, Y, test_size=0.1)
# XはIDで整数なのでint64, Yは実数値なのでfloat32のTensorに変換する
train_dataset = TensorDataset(
    torch.tensor(train_X, dtype=torch.int64), torch.tensor(train_Y, dtype=torch.float32))
test_dataset = TensorDataset(
    torch.tensor(test_X, dtype=torch.int64), torch.tensor(test_Y, dtype=torch.float32))
train_loader = DataLoader(
    train_dataset, batch_size=1024, num_workers=4, shuffle=True)
test_loader = DataLoader(
    test_dataset, batch_size=1024, num_workers=4)

説明変数Xとしてuser_id, movie_id 目的変数Yとしてratingを用意する。

In [8]:
df.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


リスト6.2　行列因子分解

ユーザーや商品のIDをK次元のベクトルに変換する方法を考えますが、これは前回の章で取り扱ったEmbeddingが適任です。

特徴ベクトルができてしまえば、あとは内積を計算するだけですので、 0〜5までに抑えるためにシグモイド関数を利用することに気をつければ以下のように記述ができます。


In [11]:
class MatrixFactorization(nn.Module):
    def __init__(self, max_user, max_item, k=20):
        super().__init__()
        self.max_user = max_user
        self.max_item = max_item
        self.user_emb = nn.Embedding(max_user, k, 0)
        self.item_emb = nn.Embedding(max_item, k, 0)

    def forward(self, x):
        user_idx = x[:, 0]
        item_idx = x[:, 1]
        user_feature = self.user_emb(user_idx)
        item_feature = self.item_emb(item_idx)

        # user_feature*item_featureは(batch_size,k)次元なので
        # kについてsumをとるとそれぞれのサンプルの内積になる
        out = torch.sum(user_feature * item_feature, 1)

        # [0, 5]の範囲に収まるように変換
        out = nn.functional.sigmoid(out) * 5
        return out

リスト6.3　ユーザーや商品の個数

In [12]:
X.max(0)

array([138493, 131262])

In [13]:
max_user, max_item = X.max(0)
# np.int64型をPythonの標準のintにキャスト
max_user = int(max_user)
max_item = int(max_item)
net = MatrixFactorization(max_user+1, max_item+1)

リスト6.4　評価関数の作成

In [14]:
def eval_net(net, loader, score_fn=nn.functional.l1_loss, device="cpu"):
    ys = []
    ypreds = []
    for x, y in loader:
        x = x.to(device)
        ys.append(y)
        with torch.no_grad():
        	ypred = net(x).to("cpu").view(-1)
        ypreds.append(ypred)
    score = score_fn(torch.cat(ys).squeeze(), torch.cat(ypreds))
    return score.item()

リスト6.5　訓練部分の作成

In [15]:
from statistics import mean

net.to("cuda:0")
opt = optim.Adam(net.parameters(), lr=0.01)
loss_f = nn.MSELoss()

for epoch in range(5):
    loss_log = []
    for x, y in tqdm.tqdm(train_loader):
        x = x.to("cuda:0")
        y = y.to("cuda:0")
        o = net(x)
        loss = loss_f(o, y.view(-1))
        net.zero_grad()
        loss.backward()
        opt.step()
        loss_log.append(loss.item())
    test_score = eval_net(net, test_loader, device="cuda:0")
    print(epoch, mean(loss_log), test_score, flush=True)

  0%|          | 0/17579 [00:00<?, ?it/s]/home/tk/dev/tokai_teio/inspections/pytorch/.venv/lib/python3.9/site-packages/torch/nn/functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
100%|██████████| 17579/17579 [00:49<00:00, 354.07it/s]


0 1.6239204300644117 0.7351387143135071


100%|██████████| 17579/17579 [00:49<00:00, 357.75it/s]


1 0.8858125946432109 0.7109100222587585


100%|██████████| 17579/17579 [00:49<00:00, 354.52it/s]


2 0.8397916714596771 0.7021937966346741


100%|██████████| 17579/17579 [00:49<00:00, 356.62it/s]


3 0.8180471800877593 0.698580801486969


100%|██████████| 17579/17579 [00:48<00:00, 359.28it/s]


4 0.806056515244441 0.6973279118537903


リスト6.6　ユーザー1の映画10に対する評価を実際に予測する場合

In [16]:
# 訓練したモデルをCPUに移す
net.to("cpu")

# ユーザー1の映画10の評価を計算したい
query = (1, 10)

# int64のTensorに変換し、batchの次元を付加
query = torch.tensor(query, dtype=torch.int64).view(1, -1)

# netに渡す
net(query)

tensor([3.3693], grad_fn=<MulBackward0>)

リスト6.7　ユーザー1の上位5 本の映画をピックアップ

In [17]:
query = torch.stack([
    torch.zeros(max_item).fill_(1),
    torch.arange(1, max_item+1)
], 1).long()

# scoresは上位k本のスコア
# indicesは上位k本の位置、すなわちmovieId
scores, indices = torch.topk(net(query), 5)

リスト6.8　非線形の関数を使用してモデリングする

In [18]:
class NeuralMatrixFactorization(nn.Module):
    def __init__(self, max_user, max_item,
                 user_k=10, item_k=10,
                 hidden_dim=50):
        super().__init__()
        self.user_emb = nn.Embedding(max_user, user_k, 0)
        self.item_emb = nn.Embedding(max_item, item_k, 0)
        self.mlp = nn.Sequential(
            nn.Linear(user_k + item_k, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, 1)
        )
        
    def forward(self, x):
        user_idx = x[:, 0]
        item_idx = x[:, 1]
        user_feature = self.user_emb(user_idx)
        item_feature = self.item_emb(item_idx)
        # ユーザー特徴量と商品特徴量をまとめて1つのベクトルにする
        out = torch.cat([user_feature, item_feature], 1)
        # まとめた特徴量ベクトルをMLPに入れる
        out = self.mlp(out)
        out = nn.functional.sigmoid(out) * 5
        return out.squeeze()

リスト6.9　区切られたジャンルをBoWに変換（<your_path>は任意のディレクトリを指定）

In [21]:
import csv
from sklearn.feature_extraction.text import CountVectorizer

# csv.DictReaderを使用してCSVファイルを読み込み
# 必要な部分だけ抽出する
with open(f"{DATA_DIR}/movies.csv") as fp:
    reader = csv.DictReader(fp)
    def parse(d):
        movieId = int(d["movieId"])
        genres = d["genres"]
        return movieId, genres
    data = [parse(d) for d in reader]

movieIds = [x[0] for x in data]
genres = [x[1] for x in data]

# データに合わせてCountVectorizerを訓練する
cv = CountVectorizer(dtype="f4").fit(genres)
num_genres = len(cv.get_feature_names())

# keyがmovieIdでvalueがBoWのTensorのdictを作る
it = cv.transform(genres).toarray()
it = (torch.tensor(g, dtype=torch.float32) for g in it)
genre_dict = dict(zip(movieIds, it))

/home/tk/dev/tokai_teio/inspections/pytorch/.venv/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


リスト6.10　カスタムDatasetの作成

In [22]:
def first(xs):
    it = iter(xs)
    return next(it)

class MovieLensDataset(Dataset):
    def __init__(self, x, y, genres):
        assert len(x) == len(y)
        self.x = x
        self.y = y
        self.genres = genres
        
        # ジャンル辞書にないmovieIdの時のダミーデータ
        self.null_genre = torch.zeros_like(
            first(genres.values()))
        
    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        x = self.x[idx]
        y = self.y[idx]
        # x = (userId, movieId)
        movieId = x[1]
        g = self.genres.get(movieId, self.null_genre)
        return x, y, g

リスト6.11　DataLoaderの作成

In [23]:
train_dataset = MovieLensDataset(
    torch.tensor(train_X, dtype=torch.int64),
    torch.tensor(train_Y, dtype=torch.float32), 
    genre_dict)
test_dataset = MovieLensDataset(
    torch.tensor(test_X, dtype=torch.int64),
    torch.tensor(test_Y, dtype=torch.float32),
    genre_dict)
train_loader = DataLoader(
    train_dataset, batch_size=1024, shuffle=True, num_workers=4)
test_loader = DataLoader(
    test_dataset, batch_size=1024, num_workers=4)

リスト6.12　ネットワークモデルの作成

In [24]:
class NeuralMatrixFactorization2(nn.Module):
    def __init__(self, max_user, max_item, num_genres,
                 user_k=10, item_k=10, hidden_dim=50):
        super().__init__()
        self.user_emb = nn.Embedding(max_user, user_k, 0)
        self.item_emb = nn.Embedding(max_item, item_k, 0)
        self.mlp = nn.Sequential(
            # num_genres分だけ次元が増える
            nn.Linear(user_k + item_k + num_genres, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, 1)
        )
        
    def forward(self, x, g):
        user_idx = x[:, 0]
        item_idx = x[:, 1]
        user_feature = self.user_emb(user_idx)
        item_feature = self.item_emb(item_idx)
        # ジャンルのBoWをcatで特徴ベクトルに結合する
        out = torch.cat([user_feature, item_feature, g], 1)
        out = self.mlp(out)
        out = nn.functional.sigmoid(out) * 5
        return out.squeeze()

リスト6.13　ヘルパー関数の修正

In [25]:
def eval_net(net, loader, score_fn=nn.functional.l1_loss, device="cpu"):
    ys = []
    ypreds = []
    # loaderはジャンルのBoWも返す
    for x, y, g in loader:
        x = x.to(device)
        g = g.to(device)
        ys.append(y)
        # userId, movieId以外にジャンルの
        # BoWもネットワーク関数に渡す
        with torch.no_grad():
        	ypred = net(x, g).to("cpu")
        ypreds.append(ypred)
    score = score_fn(torch.cat(ys).squeeze(), torch.cat(ypreds))
    return score

リスト6.14　訓練部分の作成

In [26]:
net = NeuralMatrixFactorization2(
    max_user+1, max_item+1, num_genres)
opt = optim.Adam(net.parameters(), lr=0.01)
loss_f = nn.MSELoss()

net.to("cuda:0")
for epoch in range(5):
    loss_log = []
    net.train()
    for x, y, g in tqdm.tqdm(train_loader):
        x = x.to("cuda:0")
        y = y.to("cuda:0")
        g = g.to("cuda:0")
        o = net(x, g)
        loss = loss_f(o, y.view(-1))
        net.zero_grad()
        loss.backward()
        opt.step()
        loss_log.append(loss.item())
    net.eval()
    test_score = eval_net(net, test_loader, device="cuda:0")
    print(epoch, mean(loss_log), test_score.item(), flush=True)

  0%|          | 0/17579 [00:00<?, ?it/s]/home/tk/dev/tokai_teio/inspections/pytorch/.venv/lib/python3.9/site-packages/torch/nn/functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
100%|██████████| 17579/17579 [00:56<00:00, 310.75it/s]


0 0.7540799921788659 0.6458533406257629


100%|██████████| 17579/17579 [00:55<00:00, 316.01it/s]


1 0.6899009307731886 0.6313256025314331


100%|██████████| 17579/17579 [00:55<00:00, 315.10it/s]


2 0.6695173747188653 0.6277130246162415


100%|██████████| 17579/17579 [01:01<00:00, 285.96it/s]


3 0.6567179611623135 0.6260601878166199


100%|██████████| 17579/17579 [00:56<00:00, 313.29it/s]


4 0.6397344125836869 0.6171523332595825


リスト6.15　ユーザー100に対してそれぞれのジャンルを1つだけ含んだ映画のスコアを計算

In [27]:
# 指定した位置だけ1で残りが0のTensorを返す補助関数
def make_genre_vector(i, max_len):
    g = torch.zeros(max_len)
    g[i] = 1
    return g

query_genres = [make_genre_vector(i, num_genres)
    for i in range(num_genres)]
query_genres = torch.stack(query_genres, 1)

# num_genres分だけuserId=100とmovieId=0のTensorを作って結合する
query = torch.stack([
    torch.empty(num_genres, dtype=torch.int64).fill_(100),
    torch.empty(num_genres, dtype=torch.int64).fill_(0)
], 1)

# GPUに転送
query_genres = query_genres.to("cuda:0")
query = query.to("cuda:0")

# スコアを計算
net(query, query_genres)

tensor([3.0926, 3.0952, 3.0948, 3.0926, 3.0965, 3.0956, 3.0941, 3.0938, 3.0937,
        3.0930, 3.0943, 3.0939, 3.0939, 3.0948, 3.0953, 3.0950, 3.0956, 3.0957,
        3.0930, 3.0952, 3.0957, 3.0942, 3.0934, 3.0960], device='cuda:0',
       grad_fn=<SqueezeBackward0>)